## SRJ

## Using other excitations to build the s2 state

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)





    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()


    


    @qml.qnode(dev1)
    def s2state(hf_state, ashs, params2):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i in range(len(latest_ash)):
            #print('The value of i', i)
            if len(latest_ash[i]) == 4:
                #print('The latest ash excitation in s2 state is', latest_ash[i])
                #print('params2 is', params2[i])
                qml.DoubleExcitation(params2[i], wires = latest_ash[i]) 
            elif len(latest_ash[i]) == 2:
                qml.SingleExcitation(params2[i], wires = latest_ash[i])
        return qml.state()
    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, params2, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state(hf_state, latest_ash, params2)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    def global_cost_flat(x):
        params = x[:len(ashs)]  # 0,1
        params2 = x[len(ashs):len(ashs) + 1] #2
        alpha = x[len(ashs)+1] #3
        beta = x[len(ashs)+2]  #4
        return global_cost(params, params2, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

  
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        max_values = [0, 0]         # Stores the two largest commutator values
        max_operators = [None, None] 
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:

            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The current value of', {i}, 'is', {current_value})
            
            if current_value > max_values[0]:
                max_values[1] = max_values[0]         # Shift previous max down
                max_operators[1] = max_operators[0]   
                max_values[0] = current_value
                max_operators[0] = i
            
            elif current_value > max_values[1]:
                max_values[1] = current_value
                max_operators[1] = i
                    

        print(f"The highest operator value is {max_values[0]} for operator {max_operators[0]}")  #Highest operator value
        print("Top 2 operators:", max_operators)
        print("Their commutator values:", max_values)
        old_grad.append(max_values[0])  #Appending the old gradient value
        operator_check.append(max_operators[0])
        indices_str = re.findall(r'\d+', str(max_operators[0]))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation - Taking in the second highest operator - saved in max_operator[1]
        indices_str1 = re.findall(r'\d+', str(max_operators[1]))
        excitations1 = [int(index) for index in indices_str1]
        print('The second highest operator going ', excitations1)
        latest_ash = [excitations1]
        print("Latest excitation in s2 state prep:", latest_ash)
        print('Length of latest excitation:', len(latest_ash))
        params2 = qml.numpy.array([np.pi/4], requires_grad=True)  # Initial parameters for the excitations
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Params2 are', params2)

        
        s2ostate = beta * s2state(hf_state, latest_ash, params2)
        print(qml.draw(s2state, max_length=100)(hf_state,latest_ash,params2))
        x0 = np.concatenate([params, params2, np.array([alpha]), np.array([beta])])

        print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        params = result.x[:len(ashs)]  # Extracting the first part of the result as params
        params2 = result.x[len(ashs):len(ashs) + 1]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        print('Params2 after optimization:', params2)
        alpha = result.x[len(ashs) + 1]  # Extracting the alpha parameter
        beta = result.x[len(ashs) + 2]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state(hf_state, latest_ash, params2)
        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        print('\n\n')
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=16) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668944
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582133 for operator a⁺(2) a⁺(3) a(6) a(7)
Top 2 operators: [FermiWord({(0, 2): '+', (1, 3): '+', (2, 6): '-', (3, 7): '-'}), FermiWord({(0, 0): '+', (1, 3): '+', (2, 4): '-', (3, 7): '-'})]
Their commutator values: [0.3100451588582133, 0.30781905251202657]
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
The second highest operator going  [0, 3, 4, 7]
Latest excitation in s2 state prep: [[0, 3, 4, 7]]
Length of latest excitation: 1
Params2 ar

## Old values

In [3]:
energies

[np.float64(-1.430083670661779),
 np.float64(-1.569460383551882),
 np.float64(-1.7440626583932457),
 np.float64(-1.8660667755300777),
 np.float64(-1.8758503116046774),
 np.float64(-1.8835192725165297),
 np.float64(-1.8832018741923255),
 np.float64(-1.8830798945586151),
 np.float64(-1.8834880006067307),
 np.float64(-1.8848742970715424),
 np.float64(-1.8849779254502772),
 np.float64(-1.8849121065579033),
 np.float64(-1.8849892293347668),
 np.float64(-1.8850812589590114),
 np.float64(-1.8850807692670533),
 np.float64(-1.885080769137824)]

In [4]:
old_grad

[0.3100451588582133,
 0.3467832108741954,
 0.3858930312143424,
 0.1810371916609892,
 0.1139489392171217,
 0.11979487819427365,
 0.04417499539941979,
 0.034611924673698904,
 0.04136880942224424,
 0.01971716570441913,
 0.04375237632376464,
 0.10384584951344478,
 0.009663364540754012,
 0.0006773920831285116,
 6.850836784873723e-17,
 2.376287815393632e-06]

In [5]:
ash_excitation

[[2, 3, 6, 7],
 [0, 3, 5, 6],
 [0, 1, 4, 5],
 [1, 2, 4, 7],
 [0, 4],
 [3, 7],
 [0, 3, 4, 7],
 [2, 3, 4, 5],
 [1, 2, 5, 6],
 [0, 1, 6, 7],
 [0, 2, 4, 6],
 [1, 3, 5, 7],
 [1, 5],
 [2, 6],
 [0, 6],
 [1, 7]]

## Single det for Gradient calculation - but one less parameter

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)





    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()


    


    @qml.qnode(dev1)
    def s2state(hf_state, ashs, params2):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i in range(len(latest_ash)):
            #print('The value of i', i)
            if len(latest_ash[i]) == 4:
                #print('The latest ash excitation in s2 state is', latest_ash[i])
                #print('params2 is', params2[i])
                qml.DoubleExcitation(params2[i], wires = latest_ash[i]) 
            elif len(latest_ash[i]) == 2:
                qml.SingleExcitation(params2[i], wires = latest_ash[i])
        return qml.state()
    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, params2, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state(hf_state, latest_ash, params2)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    def global_cost_flat(x):
        params = x[:len(ashs)]  # 0,1
        params2 = x[len(ashs):len(ashs) + 1] #2
        alpha = x[len(ashs)+1] #3
        beta = x[len(ashs)+2]  #4
        return global_cost(params, params2, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

  
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            if i not in operator_check:
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        latest_ash = [ashs[-1]]
        print("Latest excitation:", latest_ash)
        print('Length of latest excitation:', len(latest_ash))
        params2 = qml.numpy.array([np.pi/4], requires_grad=True)  # Initial parameters for the excitations
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Params2 are', params2)

        
        s2ostate = beta * s2state(hf_state, latest_ash, params2)
        print(qml.draw(s2state, max_length=100)(hf_state,latest_ash,params2))
        x0 = np.concatenate([params, params2, np.array([alpha]), np.array([beta])])

        print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        params = result.x[:len(ashs)]  # Extracting the first part of the result as params
        params2 = result.x[len(ashs):len(ashs) + 1]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        print('Params2 after optimization:', params2)
        alpha = result.x[len(ashs) + 1]  # Extracting the alpha parameter
        beta = result.x[len(ashs) + 2]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state(hf_state, latest_ash, params2)
        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=16) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    

H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668944
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582133 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successf

## UCCSD calculation

In [4]:
import pennylane as qml
from pennylane import numpy as np

# Define the molecule
symbols  = ['H', 'H', 'H', 'H']
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])

active_electrons = 4
charge = 0
active_orbitals = 4 

# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis = "sto-6g", charge=charge, method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals)

# Define the HF state
hf_state = qml.qchem.hf_state(active_electrons, qubits)

# Generate single and double excitations
singles, doubles = qml.qchem.excitations(active_electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

# Define the device
dev = qml.device("lightning.qubit", wires=qubits)

# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))

# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=2.0)

# Optimize the circuit parameters and compute the energy
for n in range(500):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))

step = 0,  E = -1.33256889 Ha
step = 2,  E = -1.81918930 Ha
step = 4,  E = -1.87951803 Ha
step = 6,  E = -1.88422970 Ha
step = 8,  E = -1.88479934 Ha
step = 10,  E = -1.88489488 Ha
step = 12,  E = -1.88491408 Ha
step = 14,  E = -1.88491827 Ha
step = 16,  E = -1.88491923 Ha
step = 18,  E = -1.88491945 Ha
step = 20,  E = -1.88491951 Ha
step = 22,  E = -1.88491952 Ha
step = 24,  E = -1.88491953 Ha
step = 26,  E = -1.88491953 Ha
step = 28,  E = -1.88491954 Ha
step = 30,  E = -1.88491954 Ha
step = 32,  E = -1.88491954 Ha
step = 34,  E = -1.88491955 Ha
step = 36,  E = -1.88491955 Ha
step = 38,  E = -1.88491955 Ha
step = 40,  E = -1.88491956 Ha
step = 42,  E = -1.88491956 Ha
step = 44,  E = -1.88491956 Ha
step = 46,  E = -1.88491957 Ha
step = 48,  E = -1.88491957 Ha
step = 50,  E = -1.88491957 Ha
step = 52,  E = -1.88491958 Ha
step = 54,  E = -1.88491958 Ha
step = 56,  E = -1.88491958 Ha
step = 58,  E = -1.88491959 Ha
step = 60,  E = -1.88491959 Ha
step = 62,  E = -1.88491959 Ha
step = 64,  E

## GCIM

# Cross diagonal terms eliminated

In [1]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
grad_GCIM = []  # To store the highest gradient excitation values
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886 
#r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
print(coordinates)
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        print('No of states', len(counts1))
        print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already used
                #print('The current excitation operator is', i) #Current excitation operator - fermionic one
                #print('The value of  operator check is', operator_check)  #Max excitation operator
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        grad_GCIM.append(max_value)  # Append the highest operator to the grad_GCIM list
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        states.append(ostate)
        print(f'After adding the states the adapt iteration is {j} and no of states',len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        #print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):

                #print('The value of i and j is', i, j)
                    left_op = states[i].T.conj()
                    right_op = states[j]
                    M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                    S[i,j] = left_op.dot(right_op).real
                    print(f'The value of i and j M and S is', {i}, {j}, {M[i,j]}, {S[i,j]})
                

        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        print('The shape of S matrix is', S.shape)  
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
        print('\n \n')
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state, grad_GCIM


Theta is 0.7853981633974483
[[ 0.          0.          5.66917797]
 [ 0.          0.         11.33835593]
 [ 0.          0.         17.0075339 ]
 [ 0.          0.         22.67671186]]


In [2]:
ash_excitation, states,eig,gs_energy,Ham_matrix, hf_state, grad_GCIM = adaptvqe(adapt_it=16, e_th=1e-12)
print('The final ash_excitation is', ash_excitation)
print('GS energy is', gs_energy)
print('gradient GCIM is', grad_GCIM)

HF state is -1.3325688880668922
The operator pool length is 26
The adapt iteration now is 1
The measurement of new state is {np.str_('11110000'): np.int64(1000)}
No of states 1
The operator check for this iteration 1 is []
The highest operator value is 0.3100451588582212 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
ash_excitation is [[2, 3, 6, 7]]
The length of ash_excitation before generating matrix is 1
State count after Generating function is {np.str_('11000011'): np.int64(140), np.str_('11110000'): np.int64(860)}
Skipping individual excitation state for j < 2
After adding the states the adapt iteration is 1 and no of states 2
The value of i and j M and S is {0} {0} {tensor(-1.33256889+0.j, requires_grad=True)} {tensor(1.+0.j, requires_grad=True)}
The value of i and j M and S is {0} {1} {tensor(-1.29045769+0.j, requires_grad=True)} {tensor(0.92387953+0.j, requires_grad=True)}
The value of i and j M and S is {1} {0} {tensor(-1.29045769+0.j, requires_

## Eliminating off diagonal elements

In [3]:
gs_energy

[np.float64(-1.420376125541355),
 np.float64(-1.420376125541355),
 np.float64(-1.423436434173981),
 np.float64(-1.491298935081043),
 np.float64(-1.563562006444346),
 np.float64(-1.6462342203976088),
 np.float64(-1.6999189008674949),
 np.float64(-1.7018843927346596),
 np.float64(-1.7038910905048823),
 np.float64(-1.7038910905048823),
 np.float64(-1.7038910905048823),
 np.float64(-1.7038910905048823),
 np.float64(-1.7038910905048823),
 np.float64(-1.7038910905048823),
 np.float64(-1.7038910905048823)]

In [4]:
grad_GCIM

[0.3100451588582212,
 0.3658436432869398,
 0.35472358537098203,
 0.3484777342904494,
 0.23238859022919808,
 0.27582853360402493,
 0.20855960878013377,
 0.05902197563392357,
 0.044779105494403684,
 0.07112209582106033,
 0.06399014767019294,
 0.03292761473694536,
 0.01068098935204829,
 0.009191380231021277,
 2.2089143847101673e-49]